# SpaceX Falcon 9 First Stage Landing Prediction

## EDA using SQL
* Understand the Spacex DataSet.
* Load the dataset into the corresponding table in a Db2 database.
* Execute SQL queries to answer assignment questions.

In [1]:
!pip install sqlalchemy==1.3.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Connect to the database

Loading the SQL extension to establish a connection with the database.

In [2]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [3]:
%load_ext sql
%sql sqlite:///my_data1.db

In [4]:
import pandas as pd

df = pd.read_csv('SpaceX_Data2.csv')
df.to_sql('SPACEXTBL', con, if_exists='replace', index=False, method='multi')

999

__1. Display the names of the unique launch sites in the space mission__

In [5]:
sql SELECT DISTINCT Launch_Site FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40
None


**2. Display 5 records where launch sites begin with the string 'CCA'**

In [6]:
sql SELECT * FROM SPACEXTBL WHERE Launch_Site LIKE 'CCA%' LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt


**3. Display the total payload mass carried by boosters launched by NASA (CRS)**

In [7]:
sql SELECT SUM(PAYLOAD_MASS__KG_) AS TOTAL_PAYLOAD_MASS FROM SPACEXTBL WHERE CUSTOMER = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


TOTAL_PAYLOAD_MASS
45596.0


**4. Display average payload mass carried by booster version F9 v1.1**

In [8]:
sql SELECT AVG(PAYLOAD_MASS__KG_) AS AVERAGE_PAYLOAD_MASS FROM SPACEXTBL WHERE BOOSTER_VERSION = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


AVERAGE_PAYLOAD_MASS
2928.4


**5. List the date when the first successful landing outcome in ground pad was acheived.**

In [9]:
sql SELECT MIN(DATE) AS FIRST_SUCCESSFUL_LANDING FROM SPACEXTBL WHERE LANDING_OUTCOME = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


FIRST_SUCCESSFUL_LANDING
01/08/2018


**6. List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000**

In [11]:
sql SELECT BOOSTER_VERSION FROM SPACEXTBL WHERE PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000 AND LANDING_OUTCOME = 'Success (drone ship)';

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


**7. List the total number of successful and failure mission outcomes**

In [31]:
sql SELECT MISSION_OUTCOME, COUNT(*) AS QUANTITY FROM SPACEXTBL GROUP BY MISSION_OUTCOME;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,QUANTITY
None,898
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


**8. List the names of the booster_versions which have carried the maximum payload mass. Use a subquery**

In [14]:
sql SELECT BOOSTER_VERSION FROM SPACEXTBL WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


**9. List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.**

In [28]:
sql SELECT SUBSTR(Date, 4, 2) AS MONTH, LANDING_OUTCOME, BOOSTER_VERSION, LAUNCH_SITE FROM SPACEXTBL WHERE SUBSTR(Date, 7, 4) = '2015' AND LANDING_OUTCOME = 'Failure (drone ship)';

 * sqlite:///my_data1.db
Done.


MONTH,Landing_Outcome,Booster_Version,Launch_Site
10,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


**10. Rank the count of successful landing_outcomes between the date 04-06-2010 and 20-03-2017 in descending order.**

In [30]:
sql SELECT LANDING_OUTCOME, COUNT(*) AS RANK FROM SPACEXTBL WHERE DATE BETWEEN '04-06-2010' AND '20-03-2017' GROUP BY LANDING_OUTCOME ORDER BY RANK DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,RANK
Success,20
No attempt,10
Success (drone ship),8
Success (ground pad),7
Failure (drone ship),3
Failure,3
Failure (parachute),2
Controlled (ocean),2
No attempt,1
